In [2]:
from concurrent.futures import ProcessPoolExecutor
import base64
import os

def process_image(base_64_image):
    # Split the base64 string to get the encoded part
    header, encoded = base_64_image.split(",", 1)
    data = base64.b64decode(encoded)
    
    # For demonstration, we generate a unique filename for each image
    # In a real application, you would want to manage these filenames more carefully
    filename = f'temp_image_{os.getpid()}.jpg'  # Using the process ID to generate a unique filename
    
    # Simulate a CPU-intensive task (e.g., face detection, image processing)
    # For now, we just write the data to a file
    with open(filename, 'wb') as f:
        f.write(data)
        print(f'Image processed and saved to {filename}')

    # Here you could add more CPU-intensive processing like DeepFace.extract_faces()

def add_person(images):
    # Use ProcessPoolExecutor to parallelize image processing
    with ProcessPoolExecutor(max_workers=2) as executor:
        # Map the process_image function to each image in the list
        # The results (if any) could be collected in a list
        results = list(executor.map(process_image, images))

    # Further processing or response generation goes here
    # For example, aggregating results from the processed images

# Simulate receiving a list of base64-encoded images
images = ['data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAAAAAAAD/4QB0RXhpZgAATU0AKgAAAAgAAQESAAMAAAABAAEAAIdpAAQAAAABAAAAJgAAAAAAA6ABAAMAAAABAAEAAKACAAQAAAABAAABPKADAAQAAAABAAAA7QAAAAD/2wBDAAoHBwkHBgoJCAkLCwoMDxkQDw4ODx4WFxIZJCAmJSMgI3IoLTkwKCo2KyIjMkQyNjs9QEBAJjBGS0U+Sjk/QD3/2wBDAQsLCw8NDx0QEB09KSMpPT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT3/wAARCAAoACgDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD9/KKKK/DOwooooAKKKKACiiigD//Z', 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAAAAAAAD/4QB0RXhpZgAATU0AKgAAAAgAAQESAAMAAAABAAEAAIdpAAQAAAABAAAAJgAAAAAAA6ABAAMAAAABAAEAAKACAAQAAAABAAABPKADAAQAAAABAAAA7QAAAAD/2wBDAAoHBwkHBgoJCAkLCwoMDxkQDw4ODx4WFxIZJCAmJSMgI3IoLTkwKCo2KyIjMkQyNjs9QEBAJjBGS0U+Sjk/QD3/2wBDAQsLCw8NDx0QEB09KSMpPT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT3/wAARCAAoACgDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD9/KKKK/DOwooooAKKKKACiiigD//Z']

# Call the function with the images
add_person(images)


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [17]:
import requests
import base64

# Convert image to base64
with open("../dataset/Tarmica/image_c3acccb5a39645eab99edb8d30c23144.jpg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')  # decode bytes to string

# Send the image to the server as JSON
res = requests.post(
    'https://face-id.up.railway.app/facial_recognition',
    json={'image': encoded_string, 'name': 'test'}  # use the `json` parameter instead of `data`
)

print(res.status_code)
print(res.json())


200
{'body': '"Person identified: Tarmica"', 'statusCode': 200}


In [ ]:
from deepface import Deepface
from pathlib import Path

#we need to break down the match_faces() function into 2 functions

#first function will match the faces (hint, use the verify function on cropped up picture and any of the faces in the dataset)

def crop_faces(image: Path)-> Path:
    '''extract and return paths of cropped faces'''
    extracted_faces = Deepface.extract_faces(image, enforce_detection=False)